# Wind Analysis

This notebook outlines a good example of getting started for the wind analysis. It demonstrates a couple of new, powerful tools and an example of the processing and plotting.

First, import the packages that we will be using. One new package we will be using is xarray. It is designed for working with oceanographic and other earth science datasets, particularly
netCDF, HDF5 (common in satellite/NASA products), and other multi-dimensional data.
It will also introduce matplotlib, a basic plotting library that produces nice looking graphs and figures

In [ ]:
import os, glob, re
import numpy as np
import pandas as pd
import xarray as xr
from  parse_wind_modules import parse_module

import matplotlib.pyplot as plt
# This is so figures will display in the notebook without an explicit disp(figure)
%matplotlib inline

### List of all the available .mat datasets
These datasets are the MatLab .mat files that have been parsed from the wind sensor's .DAT binary file using the matlab function decode_

In [ ]:
mat_files = sorted(glob.glob('../data/modules/**/*.mat', recursive=True))
mat_files[0:5]

In [ ]:
k = 60
filepath = mat_files[k]
print(filepath)

### Get the deployment dates
Next, we need to get the deployment dates of the cruise, because the flash cards aren't always reformatted so you can have data from previous or following cruises

In [ ]:
def get_deployment_info(array, node, deployment):
    deployment_csv = pd.read_csv(f"C:/Users/areed/Documents/OOI/oceanobservatories/ooicgsn-asset-management/deployment/{array}_Deploy.csv")
    mask = deployment_csv["deploymentNumber"] == int(deployment)
    deployment_csv = deployment_csv[mask]
    mask = deployment_csv["sensor.uid"].apply(lambda x: True if "WND" in str(x) else False)
    deployment_csv = deployment_csv[mask]
    if node == "DCL12":
        mask = deployment_csv["Reference Designator"].apply(lambda x: True if "SBD12" in x else False)
    elif node == "DCL11":
        mask = deployment_csv["Reference Designator"].apply(lambda x: True if "SBD11" in x else False)
    else:
        pass
    deployment_csv = deployment_csv[mask]
    startDateTime = deployment_csv["startDateTime"].values[0]
    stopDateTime = deployment_csv["stopDateTime"].values[0]
    return startDateTime, stopDateTime

In [ ]:
array = re.search('[A-Z]{2}[0-9]{2}[A-Z]{4}', filepath).group()
node = re.search('[a-z]{3}[1-2]{2}', filepath).group().upper()
deployment = re.search('[A-Z]{1}[0-9]{5}', filepath).group()[1:].lstrip("0")
array, node, deployment

In [ ]:
t1, t2 = get_deployment_info(array, node, deployment)
t1, t2

### Parse the .mat to a .nc file
Next, we parse the .mat file into a .nc file and save the .nc file to the same location

In [ ]:
nc_data = parse_module(filepath)
nc_data

#### Save the netCDF file

In [ ]:
nc_filepath = filepath.replace('.mat','.nc')
print(nc_filepath)
nc_data.to_netcdf(nc_filepath, format='netcdf4', engine='h5netcdf')

### Process the module data
Next, we're going to process and plot the loaded and parsed data

In [ ]:
# Filter out the time for the dataset
nc_data = nc_data.sel(time=slice(t1, t2))
nc_data

Notice that the dataset has the following "catgories": **Dimensions**, **Coordinates**, **Data variables**, **Indexes**, and **Attributes**. You can read about what each of the categories means in the [xarray documentation](https://docs.xarray.dev/en/stable/user-guide/terminology.html).

In order to reproduce the comparison of scalar wind speed against vector wind speed, we want to find which variables in the dataset represent those data. We can click on the ```Show/Hide Attributes``` button on the far side of each **Data variable** to read the _metadata_ about that variable and learn more about it. We can go ahead and click on _wspd_ variable and see that it is the Scalar averaged wind speed over one minute.

Similarly, we can click on the other data variable's attributes and learn that the other two variables we want are _wnde_, the East Wind vector value, and _wndn_, the North Wind vector value.

There remains one more step before we can get to plotting a comparison, and that is to calculate the vector wind speed average for comparison with the scalar wind speed. If we remember Calculus, we know that:

$$
\|{U}\| = \sqrt{u^{2} + v^{2}}
$$

where $\|{U}\|$ is the magnitude of the wind speed, $u$ is the eastward vector wind speed, and $v$ is the northward vector wind speed. So we can go ahead and calculate that:

In [ ]:
wvec = np.sqrt(nc_data["wnde"]**2 + nc_data["wndn"]**2)
wvec

And we can add the calculate vector wind speed to the dataset to make it easier to access and keep our workspace clean

In [ ]:
# Add the calculated results
nc_data["wvec"] = wvec
# Add some metadata so we can remember what we did
nc_data["wvec"].attrs = {
    'long_name': 'Vector Wind Speed',
    'units': 'm/s',
    'methodology': 'This is the magnitude of the east and north wind vectors'
}

In [ ]:
nc_data

Great! We now have a comparison plot. But to reproduce what is in the report, we also want to calculate the median value and standard deviation for the vector wind average based on the scalar wind speed.

In order to calculate these values, we are going to make use of the powerful [```groupby```](https://docs.xarray.dev/en/latest/user-guide/groupby.html#) method, which allows you to utilize the **split-apply-combine** strategy. Our strategy is going to be:
1. Group the dataset by the _wspd_ data. We are going to split the dataset into equally-spaced bins based on the scalar wind speed data using the ```groupby_bins``` method in ```xarray```
2. Apply calculations to the grouped data. We want the following values:
    * Mean of _wspd_
    * Median of _wvec_ ($\text{med}(U)$)
    * Mean of _wvec_ ($\bar{U}$)
    * Median standard deviation (msd) of _wvec_. This is calculated as:
$$
\text{msd} = \sqrt{ \bar{(U - \text{med}(U))^{2}} }
$$

3. Plot the data with the binned data

In [ ]:
# First, create the bins we want to group the data into
bins = np.arange(0, 31, 1)
bins

Now group the data into the bins we defined above

In [ ]:
grouped = nc_data.groupby_bins(group='wspd', bins=bins)
grouped

Calculate our statistical values

In [ ]:
# Start with mean of wspd
wspd_avg = grouped.mean()['wspd'].dropna(dim='wspd_bins')

# Next is the median of the vector wind speed
wvec_med = grouped.median()['wvec'].dropna(dim='wspd_bins')

# Finally is the mean of the vector wind speed
wvec_avg = grouped.mean()['wvec'].dropna(dim='wspd_bins')

In [ ]:
wvec_avg

In [ ]:
if 'GA01SUMO' and 'R00002' and 'dcl11' in filepath:
    nc_data = nc_data.where(nc_data['wspd'] < 24, drop=True)

In [ ]:
# To calculate the median standard deviation, we are going to have to write our own function to apply to the grouped data.
def msd(data, by, bins):
    # First, group the data
    grouped = data.groupby_bins(by, bins=bins)
    med_dif = (grouped - grouped.median())**2

    # Next, we need to group the data again
    by = by + '_bins'
    regrouped = med_dif.groupby(by)
    
    # Use those results to calculate the median standard deviation
    med_std = np.sqrt(regrouped.mean())
    
    # Return the results - this is needed to get the result out of the function
    return med_std

In [ ]:
# Calculate the median standard deviation
wvec_msd = msd(nc_data, by='wspd', bins=bins)['wvec']
wvec_msd

In [ ]:
nc_data

Yay! We now have all of the necessary data to reproduce the figure from the report 3305-000207. So lets do it.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

# Create a one:one line
x = np.arange(0, 31, 1)
y = np.arange(0, 31, 1)

# Plot the one:one line
ax.plot(x, y, color="black", linewidth=2)

# Plot the comparison
ax.plot(nc_data["wspd"], nc_data["wvec"], marker='o', linestyle='', color="red", alpha=0.3) # Alpha controls transparency, 1=solid, 0=transparent

# Plot the bins with error bars
ax.errorbar(x=wspd_avg[0:len(wvec_msd)], y=wvec_avg[0:len(wvec_msd)], yerr=wvec_msd, fmt='o', linewidth=2, capsize=6)
# Set some limits on the figure
ax.set_xlim((0,25))
ax.set_ylim((0,25))

# Add in title, axis labels, and grid lines
ax.set_title(f'{array} {node} Deployment {deployment}: S/N {nc_data.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax.set_xlabel('2-axis Sonic Winds (m/s) Scalar', fontsize=12)
ax.set_ylabel('2-axis Sonic Winds (m/s) VecAve', fontsize=12)
ax.grid()

In [ ]:
filename = filepath.split("\\")[-1].replace('.mat','.png')
filename = "_".join((array, node, 'deployment', deployment, filename))
figurepath = "../results/figures/" +  filename
figurepath

In [ ]:
fig.savefig(figurepath, facecolor="white", transparent=False, edgecolor="black", bbox_inches = "tight")

### Get the deployment start times for each deployment and parse out orders of serial numbers

In [ ]:
tracking = pd.read_excel("../data/modules/Wind_Sensor_module_tracking.xlsx")
tracking

In [ ]:
startTime = []
for idx in tracking.index:
    array = tracking.loc[idx, "Platform"]
    node = "DCL11"
    deployment = tracking.loc[idx, "Deployment"][1:].lstrip("0")
    t1, t2 = get_deployment_info(array, node, deployment)
    startTime.append(t1)

In [ ]:
tracking["deployStart"] = startTime
tracking

In [ ]:
# Split the tracking
tracking_1 = tracking[["Platform", "Deployment", "METBK1_WND", "Data Interpretation", "deployStart"]]
tracking_2 = tracking[["Platform", "Deployment", "METBK2_WND", "Data Interpretation.1", "deployStart"]]

In [ ]:
tracking_1

In [ ]:
tracking_2.rename(columns={"Data Interpretation.1":"Data Interpretation"}, inplace=True)

In [ ]:
tracking_1.rename(columns={"METBK1_WND":"Serial Number"}, inplace=True)
tracking_2.rename(columns={"METBK2_WND":"Serial Number"}, inplace=True)

In [ ]:
tracking_1["DCL"] = 11
tracking_2["DCL"] = 12

In [ ]:
tracking = pd.concat([tracking_1, tracking_2], ignore_index=True)
tracking

In [ ]:
tracking.to_excel("../data/modules/Wind_Sensor_module_tracking.xlsx", index=False)

Now we want to save the above figure. We'll do this to the same directory where we have the netCDF file. First we'll specify the filepath and create a figure name.

### Bad-Good-Bad Analysis: 233 and 243

In [ ]:
nc_files = sorted(glob.glob('../data/modules/**/SWND243.nc', recursive=True))
nc_files

In [ ]:
ossm243 = xr.open_dataset(nc_files[0]).load()
gism243 = xr.open_dataset(nc_files[1]).load()
gssm243 = xr.open_dataset(nc_files[2]).load()

In [ ]:
# Clean up the deployment times
t1, t2 = get_deployment_info('CP04OSSM', 'DCL11', 6)
ossm243 = ossm243.sel(time=slice(t1,t2))

t1, t2 = get_deployment_info('GS01SUMO', 'DCL11', 4)
t2='2019-07-28T00:00:00'
gssm243 = gssm243.sel(time=slice(t1, t2))

t1, t2 = get_deployment_info('GI01SUMO', 'DCL11', 8)
gism243 = gism243.sel(time=slice(t1, t2))

In [ ]:
def add_vector_wind(ds):
    wvec = np.sqrt(ds["wnde"]**2 + ds["wndn"]**2)
    # Add the calculated results
    ds["wvec"] = wvec
    # Add some metadata so we can remember what we did
    ds["wvec"].attrs = {
        'long_name': 'Vector Wind Speed',
        'units': 'm/s',
        'methodology': 'This is the magnitude of the east and north wind vectors'
    }
    return ds

In [ ]:
ossm243 = add_vector_wind(ossm243)
gism243 = add_vector_wind(gism243)
gssm243 = add_vector_wind(gssm243)

In [ ]:
# First, create the bins we want to group the data into
bins = np.arange(0, 31, 1)

# OSSM6
# Start with mean of wspd
grouped = ossm243.groupby_bins(group='wspd', bins=bins)
ossm243_wspd_avg = grouped.mean()['wspd'].dropna(dim='wspd_bins')

# Next is the median of the vector wind speed
ossm243_wvec_med = grouped.median()['wvec'].dropna(dim='wspd_bins')

# Finally is the mean of the vector wind speed
ossm243_wvec_avg = grouped.mean()['wvec'].dropna(dim='wspd_bins')

# Calculate the median standard deviation
ossm243_wvec_msd = msd(ossm243, by='wspd', bins=bins)['wvec']

# gism8
# Start with mean of wspd
grouped = gism243.groupby_bins(group='wspd', bins=bins)
gism243_wspd_avg = grouped.mean()['wspd'].dropna(dim='wspd_bins')

# Next is the median of the vector wind speed
gism243_wvec_med = grouped.median()['wvec'].dropna(dim='wspd_bins')

# Finally is the mean of the vector wind speed
gism243_wvec_avg = grouped.mean()['wvec'].dropna(dim='wspd_bins')

# Calculate the median standard deviation
gism243_wvec_msd = msd(gism243, by='wspd', bins=bins)['wvec']

# gssm4
# Start with mean of wspd
grouped = gssm243.groupby_bins(group='wspd', bins=bins)
gssm243_wspd_avg = grouped.mean()['wspd'].dropna(dim='wspd_bins')

# Next is the median of the vector wind speed
gssm243_wvec_med = grouped.median()['wvec'].dropna(dim='wspd_bins')

# Finally is the mean of the vector wind speed
gssm243_wvec_avg = grouped.mean()['wvec'].dropna(dim='wspd_bins')

# Calculate the median standard deviation
gssm243_wvec_msd = msd(gssm243, by='wspd', bins=bins)['wvec']

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, nrows=1, figsize=(18, 6), sharey=True)

# Create a one:one line
x = np.arange(0, 31, 1)
y = np.arange(0, 31, 1)

##### Plot the first figure
# Plot the one:one line
ax1.plot(x, y, color="black", linewidth=2)

# Plot the comparison
ax1.plot(ossm243["wspd"], ossm243["wvec"], marker='o', linestyle='', color="red", alpha=0.3) # Alpha controls transparency, 1=solid, 0=transparent

# Plot the bins with error bars
ax1.errorbar(x=ossm243_wspd_avg[0:len(ossm243_wvec_msd)], y=ossm243_wvec_avg[0:len(ossm243_wvec_msd)], yerr=ossm243_wvec_msd, fmt='o', linewidth=2, capsize=6)
# Set some limits on the figure
ax1.set_xlim((0,25))
ax1.set_ylim((0,25))

# Add in title, axis labels, and grid lines
ax1.set_title(f'CP01OSSM Deployment 6 (2017-06-06): S/N {ossm243.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax1.set_xlabel('2-axis Sonic Winds (m/s) Scalar', fontsize=12)
ax1.set_ylabel('2-axis Sonic Winds (m/s) VecAve', fontsize=12)
ax1.grid()

##### Plot the second figure
# Plot the one:one line
ax2.plot(x, y, color="black", linewidth=2)

# Plot the comparison
ax2.plot(gssm243["wspd"], gssm243["wvec"], marker='o', linestyle='', color="red", alpha=0.3) # Alpha controls transparency, 1=solid, 0=transparent

# Plot the bins with error bars
ax2.errorbar(x=gssm243_wspd_avg[0:len(gssm243_wvec_msd)], y=gssm243_wvec_avg[0:len(gssm243_wvec_msd)], yerr=gssm243_wvec_msd, fmt='o', linewidth=2, capsize=6)
# Set some limits on the figure
ax2.set_xlim((0,25))
ax2.set_ylim((0,25))

# Add in title, axis labels, and grid lines
ax2.set_title(f'GS01SUMO Deployment 4 (2018-12-04): S/N {gssm243.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax2.set_xlabel('2-axis Sonic Winds (m/s) Scalar', fontsize=12)
ax2.grid()


##### Plot the third figure
# Plot the one:one line
ax3.plot(x, y, color="black", linewidth=2)

# Plot the comparison
ax3.plot(gism243["wspd"], gism243["wvec"], marker='o', linestyle='', color="red", alpha=0.3) # Alpha controls transparency, 1=solid, 0=transparent

# Plot the bins with error bars
ax3.errorbar(x=gism243_wspd_avg[0:len(gism243_wvec_msd)], y=gism243_wvec_avg[0:len(gism243_wvec_msd)], yerr=gism243_wvec_msd, fmt='o', linewidth=2, capsize=6)
# Set some limits on the figure
ax3.set_xlim((0,25))
ax3.set_ylim((0,25))

# Add in title, axis labels, and grid lines
ax3.set_title(f'GI01SUMO Deployment 8 (2021-08-12): S/N {gism243.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax3.set_xlabel('2-axis Sonic Winds (m/s) Scalar', fontsize=12)
ax3.grid()

In [ ]:
fig.savefig("../results/SWND243_time_comparison.png", facecolor="white", transparent=False, bbox_inches="tight", edgecolor="black")

In [ ]:
from scipy.stats import circmean, circstd

In [ ]:
def circular_stats(ds):
    df = ds["compass"].to_dataframe()
    df = np.deg2rad(df)
    rolling = df.rolling(pd.Timedelta('1h'))
    avg = rolling.apply(circmean)
    std = rolling.apply(circstd)
    return avg, std

In [ ]:
gism243_avg, gism243_std = circular_stats(gism243)
ossm243_avg, ossm243_std = circular_stats(ossm243)
gssm243_avg, gssm243_std = circular_stats(gssm243)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(ncols=1, nrows=3, figsize=(18, 12))

# Create a one:one line
x = np.arange(0, 31, 1)
y = np.arange(0, 31, 1)

##### Plot the first figure
# Plot the comparison
ax1.plot(ossm243_std.index, ossm243_std["compass"], marker='o', linestyle='', color="red", alpha=0.3) # Alpha controls transparency, 1=solid, 0=transparent

# Add in title, axis labels, and grid lines
ax1.set_title(f'CP01OSSM Deployment 6 (2017-06-06): S/N {ossm243.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax1.set_ylabel('Compass Standard Error', fontsize=12)
ax1.grid()
ax1.set_ylim(0, 4.0)

##### Plot the second figure
# Plot the comparison
ax2.plot(gssm243_std.index, gssm243_std["compass"], marker='o', linestyle='', color="red", alpha=0.3) # Alpha controls transparency, 1=solid, 0=transparent

# Add in title, axis labels, and grid lines
ax2.set_title(f'GS01SUMO Deployment 4 (2018-12-04): S/N {gssm243.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax2.grid()
ax2.set_ylabel('Compass Standard Error')
ax2.set_ylim(0, 4.0)


##### Plot the third figure
# Plot the comparison
ax3.plot(gism243_std.index, gism243_std["compass"], marker='o', linestyle='', color="red", alpha=0.3) # Alpha controls transparency, 1=solid, 0=transparent

# Add in title, axis labels, and grid lines
ax3.set_title(f'GI01SUMO Deployment 8 (2021-08-12): S/N {gism243.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax3.set_xlabel('Time', fontsize=12)
ax3.set_ylabel('Compass Standard Error')
ax3.grid()
ax3.set_ylim(0, 4.0)


In [ ]:
fig.savefig("../data/

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, nrows=1, figsize=(18, 6), sharey=True)

# 
bins = np.arange(0, 4.1, 0.1)


##### Plot the first figure
# Plot the one:one line
hist, edges = np.histogram(ossm243_std["compass"], bins=bins)
freq = hist / float(hist.sum())
width = np.diff(edges) # edges is bins
ax1.bar(edges[1:], freq, width=width, align="edge", ec="k")

# Add in title, axis labels, and grid lines
ax1.set_title(f'CP01OSSM Deployment 6 (2017-06-06): S/N {ossm243.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax1.set_xlabel('Compass Standard Error\n1-hour rolling', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.grid()

##### Plot the second figure
# Plot the one:one line
hist, edges = np.histogram(gssm243_std["compass"], bins=bins)
freq = hist / float(hist.sum())
width = np.diff(edges) # edges is bins
ax2.bar(edges[1:], freq, width=width, align="edge", ec="k")

# Add in title, axis labels, and grid lines
ax2.set_title(f'GS01SUMO Deployment 4 (2018-12-04): S/N {gssm243.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax2.set_xlabel('Compass Standard Error\n1-hour rolling', fontsize=12)
ax2.grid()


##### Plot the third figure
# Plot the one:one line
hist, edges = np.histogram(gism243_std["compass"], bins=bins)
freq = hist / float(hist.sum())
width = np.diff(edges) # edges is bins
ax3.bar(edges[1:], freq, width=width, align="edge", ec="k")

# Add in title, axis labels, and grid lines
ax3.set_title(f'GI01SUMO Deployment 8 (2021-08-12): S/N {gism243.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax3.set_xlabel('Compass Standard Error\n1-hour rolling', fontsize=12)
ax3.grid()

### Get MOPAK compass data

In [ ]:
from erddapy import ERDDAP

In [ ]:
oms = ERDDAP(
    server="https://cgsn-dashboard.whoi.edu/erddap/",
    protocol="tabledap",
)

In [ ]:
search_url = oms.get_search_url(response="csv")
search = pd.read_csv(search_url)
datasets = search["Dataset ID"]
datasets

In [ ]:
mask = datasets.apply(lambda x: True if 'MOPAK' in x else False)
datasets[mask]

In [ ]:
t1, t2 = get_deployment_info('GS01SUMO', 'SBD11', 4)
t1, t2

In [ ]:
oms.dataset_id = "GS01SUMO-BUOY-MOPAK-01-1"
oms.constraints = {
    "deploy_id=": "D0004",
    "time>=": '2018-12-04T17:10:40',
    "time<=": '2020-01-20T10:01:00'
}

In [ ]:
gs01sumo_mopak = oms.to_pandas(index_col='time (UTC)', parse_dates=True)


In [ ]:
fig.savefig("../results/SWND243_compass_error_histogram.png", facecolor="white", transparent=False, edgecolor="black", bbox_inches="tight")

In [ ]:
np.

In [ ]:
gssm243["compass"].to_dataframe()

In [ ]:
# Repeat for 233
nc_files = sorted(glob.glob('../data/modules/**/SWND233.nc', recursive=True))
nc_files

In [ ]:
cnsm7 = xr.open_dataset(nc_files[0])
gism8 = xr.open_dataset(nc_files[1])
gssm4 = xr.open_dataset(nc_files[2])

In [ ]:
cnsm7 = add_vector_wind(cnsm7)
gism8 = add_vector_wind(gism8)
gssm4 = add_vector_wind(gssm4)

In [ ]:
cnsm7 = cnsm7.load()
gism8 = gism8.load()
gssm4 = gssm4.load()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, nrows=1, figsize=(18, 6), sharey=True)

# Create a one:one line
x = np.arange(0, 31, 1)
y = np.arange(0, 31, 1)

##### Plot the first figure
# Plot the one:one line
ax1.plot(x, y, color="black", linewidth=2)

# Plot the comparison
ax1.plot(cnsm7["wspd"], cnsm7["wvec"], marker='o', linestyle='', color="red", alpha=0.3) # Alpha controls transparency, 1=solid, 0=transparent

# Plot the bins with error bars
ax1.errorbar(x=cnsm7_wspd_avg[0:len(cnsm7_wvec_msd)], y=cnsm7_wvec_avg[0:len(cnsm7_wvec_msd)], yerr=cnsm7_wvec_msd, fmt='o', linewidth=2, capsize=6)
# Set some limits on the figure
ax1.set_xlim((0,25))
ax1.set_ylim((0,25))

# Add in title, axis labels, and grid lines
ax1.set_title(f'CP01CNSM Deployment 7 (2017-06-09): S/N {cnsm7.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax1.set_xlabel('2-axis Sonic Winds (m/s) Scalar', fontsize=12)
ax1.set_ylabel('2-axis Sonic Winds (m/s) VecAve', fontsize=12)
ax1.grid()

##### Plot the second figure
# Plot the one:one line
ax2.plot(x, y, color="black", linewidth=2)

# Plot the comparison
ax2.plot(gssm4["wspd"], gssm4["wvec"], marker='o', linestyle='', color="red", alpha=0.3) # Alpha controls transparency, 1=solid, 0=transparent

# Plot the bins with error bars
ax2.errorbar(x=gssm4_wspd_avg[0:len(gssm4_wvec_msd)], y=gssm4_wvec_avg[0:len(gssm4_wvec_msd)], yerr=gssm4_wvec_msd, fmt='o', linewidth=2, capsize=6)
# Set some limits on the figure
ax2.set_xlim((0,25))
ax2.set_ylim((0,25))

# Add in title, axis labels, and grid lines
ax2.set_title(f'GS01SUMO Deployment 4 (2018-12-04): S/N {gssm4.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax2.set_xlabel('2-axis Sonic Winds (m/s) Scalar', fontsize=12)
ax2.grid()


##### Plot the third figure
# Plot the one:one line
ax3.plot(x, y, color="black", linewidth=2)

# Plot the comparison
ax3.plot(gism8["wspd"], gism8["wvec"], marker='o', linestyle='', color="red", alpha=0.3) # Alpha controls transparency, 1=solid, 0=transparent

# Plot the bins with error bars
ax3.errorbar(x=gism8_wspd_avg[0:len(gism8_wvec_msd)], y=gism8_wvec_avg[0:len(gism8_wvec_msd)], yerr=gism8_wvec_msd, fmt='o', linewidth=2, capsize=6)
# Set some limits on the figure
ax3.set_xlim((0,25))
ax3.set_ylim((0,25))

# Add in title, axis labels, and grid lines
ax3.set_title(f'GI01SUMO Deployment 8 (2021-08-12): S/N {gism8.attrs["serial_number"]}', fontsize=14) # f-strings are a way to substitute a variable into a string and have it print out
ax3.set_xlabel('2-axis Sonic Winds (m/s) Scalar', fontsize=12)
ax3.grid()

In [ ]:
fig.savefig("../results/SWND233_time_comparison.png", facecolor="white", transparent=False, bbox_inches="tight", edgecolor="black")

In [ ]:
# Read in the data
# Record data:        hr min sec day dow mon year wnde wndn wspd spdmax lxydir compass tiltx tilty svel temp end
#        bytes:       1  1   1   1   1   1   2    2    2    1    1      2      2       1     1     4    4    4
#        format:      B  B   B   B   B   B   <H   <h   <h   B    B      <h     <h      b     b     <f   <f   